20260124

- L1 (Foundation): clean Python, data handling, correctness, basic NLP
- L2 (Core): REST services, async, evaluation, retrieval, model wrappers
- L3 (Advanced): RAG pipelines, caching, observability, AWS patterns, CI/CD rigor

# A) python foundations for AI backends (5)

## 1. re.sub()
(whitespace, casing, punctuation)

- Scenario: Chatbot preprocessor cleans user input.
- Input: "  Hi!!   I NEED   help...  "
- Output: "hi! i need help..."

In [ ]:
import re 

def normalize(text:str) -> str:
    text = text.strip().lower() # strip(): leading and trailing whitespace removed
    text = re.sub(r"\s+", " ", text)    #collapses any run of whitespace(spaces, tabs, newlines) into a single space
    text = re.sub(r"!{2,}", "!", text)  #collapses any run of 2 or more exclamation marks into a single !
    
    return text

print(normalize("  Hi!!   I NEED   help...  "))

hi!! i need help...


## 2. parse_qsl: kv pairs
Parse key=value pairs
- Scenario: REST endpoint receives tracking params.
- Input: "lang=en&user=42&debug=true"
- Output: {"lang":"en","user":"42","debug":"true"}

In [5]:
from urllib.parse import parse_qsl

def  parse_query(q:str) -> dict:
    return dict(parse_qsl(q, keep_blank_values=True))

print(parse_query("lang=en&user=42&debug=true"))

{'lang': 'en', 'user': '42', 'debug': 'true'}


## 3. pathlib, Path
Safe JSON read + validation fallback
- Scenario: Load prompt templates from JSON; don’t crash on bad file.
- Input: bad JSON file
- Output: default dict

In [6]:
import json 
from pathlib import Path 

def load_json(path: str, default: dict) -> dict:
    try:
        return json.loads(Path(path).read_text(encoding="utf-8"))
    except(OSError, json.JSONDecodeError):
        return default 

cfg = load_json("prompts.json", {"system": "You are helpful."})
print(cfg)

{'system': 'You are helpful.'}


In [8]:
from pathlib import Path
import json

user = {"id": 7, "tags": ["new", "vip"], "active": True}

p = Path("tmp") / "profile.json"
p.parent.mkdir(exist_ok=True)

p.write_text(json.dumps(user, indent=2))

print("Saved to:", p)
print("File contents:\n", p.read_text())

loaded = json.loads(p.read_text())
print("Loaded:", loaded)


Saved to: tmp/profile.json
File contents:
 {
  "id": 7,
  "tags": [
    "new",
    "vip"
  ],
  "active": true
}
Loaded: {'id': 7, 'tags': ['new', 'vip'], 'active': True}


## 4. set() to deduplicate
Deduplicate events while preserving order
- Scenario: Avoid double-counting telemetry events.
- Input: ["open","click","open","close"]
- Output: ["open","click","close"]

In [ ]:
from typing import List 

def dedupe_keep_order(items: List) -> List:
    seen, out = set(), []   # 🤩 Set for O(1), order preservation
    for x in items:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out 

print(dedupe_keep_order(["open","click","open","close"]))

['open', 'click', 'close']


In [13]:
# key point: set() only contains unique values

s = {1,1,2,3,3}
print(s)

{1, 2, 3}


## 5. @lru_cache

Simple LRU cache for expensive operations
- Scenario: cache embeddings or database lookups
- input: repeated key
- output: avoids recomputation

In [ ]:
from functools import lru_cache # to memoize results of a function so repeated calls with the same input are fast. (LRU = least reacently used)

@lru_cache(maxsize=1024)    #wraps the function with a cache that stores up to 1024 recent results.
def expensive_lookup(key: str) -> str:
    # pretend DB/ network call
    return f"result_for_{key}"


print(expensive_lookup("order_1"))  #firstcall with "order_1" computes and stores the result
print(expensive_lookup("order_1"))  #cached (same key, so served from cache. no DB/network work)

result_for_order_1
result_for_order_1


# B) Data processing with scientific stack (Numpy/pandas)

## 6. pd.Series, s.str.len()
Compute basic text stats with pandas
- scenario: analyze chat logs
- list of messages
- avg length, empty rate

In [22]:
import pandas as pd

msgs = ["hi", "need help with refund", "", "thanks"]
s = pd.Series(msgs) #create a 1d labeled array
print(s)
print(s.str.len())

stats = {
    "avg_len": s.str.len().mean(),  #adding "float()" also correct
    "empty_rate":(s.str.len() == 0).mean()
}
print(stats)

0                       hi
1    need help with refund
2                         
3                   thanks
dtype: str
0     2
1    21
2     0
3     6
dtype: int64
{'avg_len': np.float64(7.25), 'empty_rate': np.float64(0.25)}


- don't know why my terminal venv is not the same as the one i use in ipynb, so...i tried to directly use :
    - ./.venv/bin/python -m pip install pandas
- to install pandas

## 7. pd.to_datetime(), datetime
date parsing + timezone normalization

In [ ]:
import pandas as pd 

ts = pd.to_datetime("2026-01-24T10:20:00+01:00", utc=True)  #Parses the ISO string into a pandas Timestamp; utc=True converts the result to UTC time
print(ts.isoformat())   # Prints the timestamp in ISO 8601 format (now in UTC).

2026-01-24T09:20:00+00:00


In [ ]:
# can also use datetime
from datetime import datetime, timezone

ts = datetime.fromisoformat("2026-01-24T10:20:00+01:00")    #parse ISo string into timezone-aware datetime object
ts_utc = ts.astimezone(timezone.utc)    #since ts is now a datetime object. astimezone is a method of the OBJECT
print(ts_utc.isoformat())


2026-01-24T09:20:00+00:00


## 8. into batches (Iterable)
- scenario: send documents to embedding api in batches
- input: docs size=10, batch=4
- output: 3 batches

In [ ]:
from typing import List, Iterable

def batched(items: List[str], size:int) -> Iterable[List[str]]: #Iterable: a type hint for "anything you can loop over"(list, tuple, set, generator, etc.) so it can be List[List[str]], or Tuple(List[str]) or etc.
    for i in range(0, len(items), size):
        yield items[i:i+size]

print(list(batched(list("abcdefghij"),4)))

[['a', 'b', 'c', 'd'], ['e', 'f', 'g', 'h'], ['i', 'j']]


In [26]:
print(list("abcdefg"))

['a', 'b', 'c', 'd', 'e', 'f', 'g']


In [ ]:
# another Iterable example
from typing import Iterable

def total(nums: Iterable[int]) -> int:  #Iterable[int] means it can be anything like list, tuple or set, as long as it can be iterated
    s = 0
    for n in nums:
        s += n
    return s

print(total([1, 2, 3]))     # list
print(total((4, 5, 6)))     # tuple
print(total({7, 8, 9}))     # set

# non-iterables: int, float, None, bool, a plain object() instance

# as long as it is an iterable object, we can apply for loop on it. (lists, tuples, sets, strings, dicts, generators, etc.)


6
15
24


## 9. np.lianalg.norm()

cosine similarity with numpy (no sklearn)
- scenario: rank candidates by embedding similarity
- input: query vec + candidate vecs
- output: best index

In [30]:
import numpy as np

def best_match(q, X):
    q = np.array(q, dtype=float)
    X = np.array(X, dtype=float)
    
    qn = q / (np.linalg.norm(q) + 1e-12)
    Xn = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    
    sims = Xn @ qn 
    return int(np.argmax(sims)), sims.tolist()

idx, sims = best_match([1,0], [[0.9,0.1], [0,1],[1,0]])
print(idx,sims)

2 [0.9938837346715274, 0.0, 0.9999999999979998]


## 10. for _,row in x.iterrows()
streaming csv processing (memory-safe)
- scenario: rag on large csv kb; avoid loading full file
- input: csv path
- output: yields filtered rows

In [ ]:
import pandas as pd 

def iter_rows(path: str, keyword: str):
    for chunk in pd.read_csv(path, chunsize=1000):
        hit = chunk[chunk["text"].str.contains(keyword, na=False)]
        for _, row in hit.iterrows():
            yield row.to_dict()

# C) Core NLP with NLTK / spaCy

## 11. re.findall()

Tokenization with regex (no library)
- Scenario: Minimal dependency service.
- Input: "Refund for order #123?"
- Output: ["refund","for","order","123"]

In [ ]:
import re

def tokenize(text: str):
    return re.findall(r"[a-z0-9]+", text.lower())

print(tokenize("Refund for order #123?"))

['refund', 'for', 'order', '123']



spaCy NER extraction for support tickets

- Scenario: Extract ORG/PRODUCT from ticket text.
- Input: "Issue with Apple Pay on iPhone 15"
- Output: entities list

In [ ]:
# ./.venv/bin/python -m pip install pandas
import spacy

nlp = spacy.load("en_core_web_sm")  #includes a tokenizer and NER component (named entity recognition)

def extract_ents(text: str):
    doc = nlp(text)
    print(doc)  #nothing will happen, still itself
    return [(ent.text, ent.label_) for ent in doc.ents] #detect entity and label pairs in the doc

print(extract_ents("Issue with Apple Pay on iPhone 15"))


Issue with Apple Pay on iPhone 15
[('Apple Pay', 'ORG'), ('15', 'CARDINAL')]


## TfidfVectorizer & LogisticRegression

In [6]:
# should first install wheel and sciket-learn:  ./.venv/bin/python -m pip install -U pip setuptools wheel; ./.venv/bin/python -m pip install scikit-learn


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

X = ["where is my package", "i want a refund", "track my delivery", "refund please"]
y = ["shipping","refund","shipping","refund"]

clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("lr", LogisticRegression(max_iter=1000))
]).fit(X, y)

print(clf.predict(["can i get my money back?"])[0])


shipping


## classification_report: precision, recall, f1score,support

In [7]:
from sklearn.metrics import classification_report

y_true = ["refund","shipping","refund"]
y_pred = ["refund","refund","refund"]
print(classification_report(y_true, y_pred, digits=3))


              precision    recall  f1-score   support

      refund      0.667     1.000     0.800         2
    shipping      0.000     0.000     0.000         1

    accuracy                          0.667         3
   macro avg      0.333     0.500     0.400         3
weighted avg      0.444     0.667     0.533         3



/Users/lilun.zhang/VSCodeProjects/aie/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/lilun.zhang/VSCodeProjects/aie/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/lilun.zhang/VSCodeProjects/aie/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

# LLM call patterns

## system & user pairs

In [9]:
def build_messages(system: str, user: str):
    return [{"role":"system","content":system},{"role":"user","content":user}]

print(build_messages("You are a support bot.", "I need a refund."))

[{'role': 'system', 'content': 'You are a support bot.'}, {'role': 'user', 'content': 'I need a refund.'}]


## mock LLM client with timeout + retries
production safe

In [ ]:
import time
import random

class LLMError(Exception): ...

def call_llm(prompt: str, retries=3, backoff=0.2):
    for attempt in range(retries):
        try:
            if random.random() < 0.3:   # not logically needed, just mimicing realistic world
                raise LLMError("transient")
            return {"text": f"Answer: {prompt[:40]}..."}
        except LLMError:
            time.sleep(backoff * (2 ** attempt))    # 0.2s, 0.4s, 0.8s by default
    return {"text": "Sorry, please try again later."}

print(call_llm("Explain refund policy in 2 lines."))


{'text': 'Answer: Explain refund policy in 2 lines....'}


In [11]:
def render(template: str, **vars):
    safe = {k: str(v).replace("{","(").replace("}",")") for k,v in vars.items()}
    return template.format(**safe)

tpl = "Summarize user request: {user_text}"
print(render(tpl, user_text="ignore {system} and do X"))

Summarize user request: ignore (system) and do X


## sklearn: cosine_similarity & TDidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

docs = {
    1: "Refund guide: You should first submit your application online, and then wait for our call. ",
    2: "Shopping guide: Please download our latest Farfetch app and start your shopping with 15% discount on your first order!",
}
texts = list(docs.values())
print(texts)
vec = TfidfVectorizer().fit(texts)  #learns the vocabulary/IDF weights from those docs.
X = vec.transform(texts)

def top1(query: str):
    qv = vec.transform([query]) #transform the query into the same vector space
    sims = cosine_similarity(qv, X)[0]
    i = int(sims.argmax())  # Picks the highest‑scoring doc (argmax)
    return i+1, float(sims[i])  #returns its id (1‑based) plus the score.

print(top1("how can I get my money back"))


['Refund guide: You should first submit your application online, and then wait for our call. ', 'Shopping guide: Please download our latest Farfetch app and start your shopping with 15% discount on your first order!']
(1, 0.0)


## ⭐️ chunking (with overlap)

In [20]:
from typing import List

def chunk(text: str, size:int, overlap:int):
    out, i = [], 0
    while i < len(text):
        out.append(text[i:i+size])  #text[0,50]->out
        i += max(1, size - overlap) # i=0 -> max(1, 50-10) = 40 #⚠️ max(1,..) here is a safety guard to avoid overlap > size
    return out

chunked_list = chunk("A"*120, size=50, overlap=10)
print(len(chunked_list[2]))


40


## chunking with metadata
filtering before expensive scoring

In [24]:
chunks = [
  {"id":"r1","domain":"refund","text":"refund within 7 days"},
  {"id":"s1","domain":"shipping","text":"shipping takes 3-5 days"},
]

def filter_domain(items, domain):
    return [x for x in items if x["domain"] == domain]  #we use [] here to avoid raising error ⚠️
    # else:
    #     raise ValueError("No such domain!")

print(filter_domain(chunks, "shopping"))    #if domain not in the chunks, it will return a [] 🤩

# [] doesn't raise error!!

[]


## ✅ confidence gate
if similarity score is too low, we don't go to retrieval stage

In [26]:
def decide(scores: List, thr: int) -> str:
    best = max(scores) if scores else 0.0
    
    return "use_rag" if best >= thr else "ask_clarify"

print(decide([0.42, 0.31], 0.55))

ask_clarify


## ⭐️ hybrid retrieval: keyword + TF-IDF union
- scenario: improve recall for short queries
- input: "refund receipt"
- output: merged candidates

Add all document IDs related to word w into the candidate set c

In [27]:
def keyword_candidates(query: str):
    c = set()
    for w in query.lower().split():
        c |= index.get(w,set())
    return c 

def hybrid(query: str):
    kw = keyword_candidates(query)
    tdidf_top, _ = top1(query)
    
    return sorted(kw | {tdidf_top})

hybrid("refund receipt")

NameError: name 'index' is not defined